In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics  import confusion_matrix, classification_report

from sklearn import svm
from matplotlib import pyplot as plt
import seaborn as sns  
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
%matplotlib inline

In [2]:
df = pd.read_csv('WineQT.csv') #  მონაცემების წაკითხვა
df.head() # პირველი 5 ჩანაწერის ნახვა

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [3]:
df.info() # მონაცემების გადახედვა

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
 12  Id                    1143 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 116.2 KB


In [4]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
count,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000
mean,8.311111,0.531339,0.268364,2.532152,0.086933,15.615486,45.914698,0.996730,3.311015,0.657708,10.442111,5.657043,804.969379
std,1.747595,0.179633,0.196686,1.355917,0.047267,10.250486,32.782130,0.001925,0.156664,0.170399,1.082196,0.805824,463.997116
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000,0.000000
25%,7.100000,0.392500,0.090000,1.900000,0.070000,7.000000,21.000000,0.995570,3.205000,0.550000,9.500000,5.000000,411.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,37.000000,0.996680,3.310000,0.620000,10.200000,6.000000,794.000000
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,61.000000,0.997845,3.400000,0.730000,11.100000,6.000000,1209.500000
max,15.900000,1.580000,1.000000,15.500000,0.611000,68.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000,1597.000000


In [5]:
df.isnull().sum() # ვამოწმებთ ცარიელი მნიშვნელობების რაოდენობას

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
Id                      0
dtype: int64

In [6]:
bins = (2,6,8) # მონაცემების გამოყოფი რიცხვითი შუალედები
group_names = ['bad', 'good'] # კატეგორიების სახელები
df['quality'] = pd.cut(df['quality'], bins = bins, labels = group_names) # კატეგორიებში დაყვანა
# ორჯერ არ გაეშვება ეს ბრძანება რადგან უკვე გავაკეთეთ 

In [7]:
df['quality'].unique() # შემოწმება რამდენი კატეგორიაა და როგორ განსაზღვრავს
df['quality'].value_counts() # რამდენი ელემენტია თითოეულ კატეგორიაში

quality
bad     984
good    159
Name: count, dtype: int64

In [8]:
label_quality = LabelEncoder()
df['quality'] = label_quality.fit_transform(df['quality']) # კატეგორიების გადაყვანა რიცხვებად

In [9]:
x = df.drop('quality',axis=1) # მახასიათებელი ცვლადები 
y = df['quality'] # სამიზნე ცვლადი
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42) # მონაცემების გაყოფა სასწავლებლად 

* მონაცემთა კლასიფიკაცია

In [10]:
sc = StandardScaler() # სტანდარტიზაცია
X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test)


* Random Forrest კლასიფიკატორი მოდელის შემქნა, დასწავლა, პროგნოზირება 

In [11]:
# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train,y_train)
pred_rfc = rfc.predict(X_test)

* მოდელის მიერ პროგნოპირებული მონაცემთა მნიშვნელობები (ღვინის ხარისხი)

In [12]:
pred_rfc[:] # პრედიქტირებული მონაცემები

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
print(classification_report(y_test,pred_rfc)) # შედეგების გამოტანა
print(confusion_matrix(y_test,pred_rfc)) # შედეგების გამოტანა

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       201
           1       0.76      0.57      0.65        28

    accuracy                           0.93       229
   macro avg       0.85      0.77      0.81       229
weighted avg       0.92      0.93      0.92       229

[[196   5]
 [ 12  16]]


* Supported Vector Machine კლასიფიკატორი მოდელის შემქნა, დასწავლა, პროგნოზირება 

In [16]:
svc = svm.SVC()
svc.fit(X_train,y_train)
pred_svc = svc.predict(X_test)

In [17]:
print(classification_report(y_test,pred_svc)) # შედეგების გამოტანა
print(confusion_matrix(y_test,pred_svc)) # შედეგების გამოტანა

              precision    recall  f1-score   support

           0       0.91      0.98      0.94       201
           1       0.69      0.32      0.44        28

    accuracy                           0.90       229
   macro avg       0.80      0.65      0.69       229
weighted avg       0.89      0.90      0.88       229

[[197   4]
 [ 19   9]]


* Neural Network კლასიფიკატორი მოდელის შემქნა, დასწავლა, პროგნოზირება 

In [18]:
mlpc = MLPClassifier(hidden_layer_sizes=(11,11,11), max_iter=500)
mlpc.fit(X_train, y_train)
mlpc_pred = mlpc.predict(X_test)

/home/metaphysicist/Coding/Projects/ML_2024/.ml/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [19]:
print(classification_report(y_test,mlpc_pred)) # შედეგების გამოტანა
print(confusion_matrix(y_test,mlpc_pred)) # შედეგების გამოტანა

              precision    recall  f1-score   support

           0       0.95      0.93      0.94       201
           1       0.56      0.64      0.60        28

    accuracy                           0.90       229
   macro avg       0.76      0.79      0.77       229
weighted avg       0.90      0.90      0.90       229

[[187  14]
 [ 10  18]]
